<a href="https://colab.research.google.com/github/sakethnarayana/cyber-bully-text-bi-lstm/blob/main/prefinal_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import functools, re
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from textblob import TextBlob
from pydub import AudioSegment
import speech_recognition as sr

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Load the model
model =  tf.keras.models.load_model('/content/drive/MyDrive/Project/models/cyberbullying_model_bilstm.h5')
#load the dataset
df = pd.read_csv("/content/drive/My Drive/Project/twitter.csv")
stopwords = set(stopwords.words("english"))
stopwords = [i.lower() for i in nltk.corpus.stopwords.words('english') + [chr(i) for i in range(97, 123)]]
x = df.tweet_text.apply(lambda text: re.sub("\s+", " ", ' '.join([i for i in re.sub("[^9A-Za-z ]", "" , re.sub("\\n", "", re.sub("\s+", " ", re.sub(r'http\S+', '', text.lower())))).split(" ") if i not in stopwords]))).values.astype(str)
y = pd.get_dummies(df.cyberbullying_type)
labels = list(y.columns)

In [ ]:
choice=input("audio or text : ")
if choice=='audio':
  # Load the MP3 file
  filename=input("enter audio file name:")
  mp3_audio = AudioSegment.from_file("/content/drive/My Drive/audios/"+filename+".mp3", format="mp3")

  # Save the MP3 file as a WAV fil
  mp3_audio.export("audio.wav", format="wav")

# Transcribe the WAV file to text 
  r = sr.Recognizer()
  audio_file = sr.AudioFile("audio.wav")

  with audio_file as source:
      audio_text = r.record(source)

  result = r.recognize_google(audio_text, show_all=True)
  custom_input= result['alternative'][0]['transcript']
  print(custom_input)
else:
  custom_input=str(input("Enter text to check whether it is bully or not"))


def sentiment_analysis(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 1
    else:
        return 0

def fun(arr):
  if arr in ['other_cyberbullying','gender','age','ethnicity','religion']:
    return 0
  else:
    return 1
    
x=sentiment_analysis(custom_input)
tokenizer=Tokenizer()

# Preprocess the custom input text in the same way as you did for the training data
custom_input = re.sub("\s+", " ", ' '.join([i for i in re.sub("[^9A-Za-z ]", "" , re.sub("\\n", "", re.sub("\s+", " ", re.sub(r'http\S+', '', custom_input.lower())))).split(" ") if i not in stopwords]))
custom_input = tokenizer.texts_to_sequences([custom_input])
custom_input = pad_sequences(custom_input, maxlen=100, padding='post', truncating='post')

# Make the prediction using the model
prediction = model.predict(custom_input)

# Get the index of the highest predicted class
predicted_class_index = np.argmax(prediction)

# Get the corresponding label for the predicted class
predicted_class = labels[predicted_class_index]

# print("The predicted cyberbullying type is:", fun(predicted_class) or x )
if fun(predicted_class) or x :
  print("the text is not cyber bullying")
else:
  print("the text is cyber bullying")


audio or text : text
Enter text to check whether it is bully or notyou are ugly
1/1 [==============================] - 0s 50ms/step
the text is cyber bullying
